In [1]:
""" 
Fine tune a image captioning or description model on a artwork description data to
obtain a model that is able to analyze an artwork and generate a description of 
the artwork that is hopefully more art-review-like than a generic image captioning 
model would

The pretrained model used here is the Salesforce Blip Captioniong model: 
https://huggingface.co/Salesforce/blip-image-captioning-base

The fine tuning data set is the data from the SemArt Project: 
https://github.com/noagarcia/SemArt
"""

' \nFine tune a image captioning or description model on a artwork description data to\nobtain a model that is able to analyze an artwork and generate a description of \nthe artwork that is hopefully more art-review-like than a generic image captioning \nmodel would\n\nThe pretrained model used here is the Salesforce Blip Captioniong model: \nhttps://huggingface.co/Salesforce/blip-image-captioning-base\n\nThe fine tuning data set is the data from the SemArt Project: \nhttps://github.com/noagarcia/SemArt\n'

In [2]:
import os
import torch
import pandas as pd
import chardet
from PIL import Image
from torchvision import transforms
from datasets import Dataset, DatasetDict
import pandas as pd
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import datasets
from datasets import Dataset as HFDataset, DatasetDict, IterableDataset
from transformers import BlipProcessor, BlipForConditionalGeneration, ViTImageProcessor, VisionEncoderDecoderModel, AutoTokenizer, TrainingArguments, Trainer

In [3]:
def list_files_in_directory(dir_path):
    try:
        files = os.listdir(dir_path)
        return [x for x in files if '.jpg' in x]
    except FileNotFoundError:
        return f"Directory not found: {dir_path}"
    except NotADirectoryError:
        return f"Not a directory: {dir_path}"

In [4]:
import pandas as pd
import torch
from torchvision import transforms
from PIL import Image
import os

# Step 1: Load the CSV file as a dictionary mapping image names to descriptions
semart_dir = f"/Users/rckyi/Documents/Datasette/SemArtData/SemArt" 
images_dir = semart_dir 

description_file_train = semart_dir + '/semart_train.csv'
description_file_test = semart_dir + '/semart_test.csv'

with open(description_file_train, 'rb') as file:
    print(f'file path {description_file_train}')
    result = chardet.detect(file.read())
    encoding = result['encoding']
    print(f'encoding {encoding}')
    df_train = pd.read_csv(description_file_train, encoding=encoding, sep='\t')
    print(type(df_train['IMAGE_FILE'][0]))
# df_test = pd.read_csv(description_file_test, encoding = "utf-8")


image_to_description = dict(zip(df_train["IMAGE_FILE"], df_train["DESCRIPTION"]))
# list(image_to_description.items())[0:5]

# # Define the image directory and transformation
# image_dir = "path/to/images"
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to a fixed shape
    transforms.ToTensor(),  # Convert image to tensor
#     transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize
])


file path /Users/rckyi/Documents/Datasette/SemArtData/SemArt/semart_train.csv
encoding utf-8
<class 'str'>


In [5]:
# # Step 1: Load the CSV file into a dictionary mapping image names to descriptions
# description_file = "path/to/description.csv"
# df = pd.read_csv(description_file)
image_files_in_dir = list_files_in_directory(semart_dir+'/Images/')
# image_files_in_dir
# list(image_to_description.items())[0]

In [6]:
# def data_generator(images_name_list):
#     image_file_list = list(df_train['IMAGE_FILE'])
#     for image_name in images_name_list:
# #         image_path = os.path.join(images_dir, image_name)
#         image_path = f"{semart_dir}/Images/{image_name}"

#         if image_name in image_file_list: #image_to_description: # and os.path.exists(image_path):
#             image = Image.open(image_path).convert("RGB")
#             image_tensor = transform(image)
#             yield image_name, image_tensor, image_to_description[image_name]

# # Step 3: Iterate through the generator and build the dictionary
# data_var = {img_name: [img_tensor, desc] for img_name, img_tensor, desc in data_generator(image_files_in_dir)}


In [7]:
from datasets import IterableDataset, DatasetDict
from transformers import BlipProcessor, BlipForConditionalGeneration, TrainingArguments, Trainer
import torch

def data_generator(images_name_list):
    ret = []
    image_file_list = list(df_train['IMAGE_FILE'])
#     def generator():
    for image_name in images_name_list:
#         image_path = os.path.join(images_dir, image_name)
        image_path = f"{semart_dir}/Images/{image_name}"

        if image_name in image_file_list: #image_to_description: # and os.path.exists(image_path):
            image = Image.open(image_path).convert("RGB")
            image_tensor = transform(image)
            ret.append({"image":image_tensor, "text":image_to_description[image_name]})
#             yield {"image":image_tensor, "text":image_to_description[image_name]}
    return ret

        
#     # Use IterableDataset.from_generator() for streaming support
#     hfd = IterableDataset.from_generator(data_generator)
#     return hfd # DatasetDict({"train": hfd})

# Step 3: Iterate through the generator and build the dictionary
# y = data_generator(images_name_list=image_files_in_dir)
# data_var = []
# for x in data_generator(images_name_list=image_files_in_dir):
#     data_var.append({"image": x["image"], "text": x["text"]} )

In [8]:
data_var = data_generator(image_files_in_dir[0:10])

In [9]:
print(data_var)

[{'image': tensor([[[0.0392, 0.0078, 0.0039,  ..., 0.0549, 0.0510, 0.0784],
         [0.1333, 0.0471, 0.0275,  ..., 0.1216, 0.0784, 0.0824],
         [0.1412, 0.1176, 0.1569,  ..., 0.0745, 0.0745, 0.0902],
         ...,
         [0.2941, 0.2627, 0.1922,  ..., 0.2078, 0.1569, 0.1961],
         [0.2863, 0.2588, 0.2039,  ..., 0.2510, 0.2196, 0.2314],
         [0.2392, 0.2510, 0.2275,  ..., 0.3255, 0.3059, 0.3216]],

        [[0.0431, 0.0157, 0.0078,  ..., 0.0902, 0.0863, 0.1137],
         [0.1294, 0.0431, 0.0235,  ..., 0.1569, 0.1137, 0.1176],
         [0.1255, 0.1020, 0.1412,  ..., 0.1098, 0.1098, 0.1255],
         ...,
         [0.2118, 0.1843, 0.1255,  ..., 0.1804, 0.1373, 0.1686],
         [0.1922, 0.1725, 0.1255,  ..., 0.2039, 0.1922, 0.2039],
         [0.1608, 0.1765, 0.1569,  ..., 0.2706, 0.2667, 0.2745]],

        [[0.0235, 0.0039, 0.0039,  ..., 0.0235, 0.0196, 0.0471],
         [0.0980, 0.0235, 0.0157,  ..., 0.0902, 0.0510, 0.0510],
         [0.0863, 0.0667, 0.1176,  ..., 0.0471,

In [10]:
hf_dataset = DatasetDict({"train": datasets.Dataset.from_list(data_var)})
hf_dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'text'],
        num_rows: 10
    })
})

In [13]:
def transforms(example_batch):
#     ((V + 1.0) * 255/2).astype(np.uint8)
    images = np.asarray([x for x in example_batch["image"]])
    captions = [x for x in example_batch["text"]]
    inputs = processor(images=images, text=captions, padding="max_length")
    inputs.update({"labels": inputs["input_ids"]})
    return inputs

hf_dataset.set_transform(transforms)

In [14]:
hf_dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'text'],
        num_rows: 10
    })
})

In [15]:
# Define a custom collator to handle the format expected by the BLIP model
def collate_fn(batch):
#     print(f'batch {batch}')
    images = [item['image'] for item in batch]
    texts = [item['text'] for item in batch]
    # Convert the images into tensor and normalize
    images = torch.stack(images)
    images = torch.add(images,1.0)
    text = torch.stack(texts)
    return processor(images=images, text=texts, padding=True, return_tensors="pt")


In [23]:
# Define Training Arguments
training_args = TrainingArguments(
    output_dir="./blip-finetuned",
    per_device_train_batch_size=8,
    num_train_epochs=5,
    save_steps=500,
    evaluation_strategy=None,
#     evaluation_strategy="steps",
    save_total_limit=2,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    push_to_hub=False,  # Set to True if uploading to Hugging Face Hub
    logging_steps=10,
    fp16=torch.cuda.is_available(),
    remove_unused_columns=False,
    max_steps=10,
    no_cuda=True,
)

TypeError: __init__() got an unexpected keyword argument 'do_rescale'

In [24]:
# Load Pretrained BLIP Model & Processor
model_name = "Salesforce/blip-image-captioning-base"
processor = BlipProcessor.from_pretrained(model_name)
model = BlipForConditionalGeneration.from_pretrained(model_name)

In [19]:
# # Define Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=hf_dataset["train"].with_format("torch"),
#     tokenizer=processor,
# )

In [25]:
# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= hf_dataset["train"],
    tokenizer=processor,
#     data_collator=collate_fn,  # Use custom collate function to prepare batches
)

<ipython-input-25-08206ab134dc>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
max_steps is given, it will override any value given in num_train_epochs


In [26]:
# hf_dataset["train"]

In [27]:
# Start Fine-Tuning
trainer.train()


Step,Training Loss
10,9.469500


TrainOutput(global_step=10, training_loss=9.469465637207032, metrics={'train_runtime': 232.8653, 'train_samples_per_second': 0.344, 'train_steps_per_second': 0.043, 'total_flos': 2.96711826112512e+16, 'train_loss': 9.469465637207032, 'epoch': 5.0})